# Imports

In [5]:
import datetime 
import pandas as pd

## Read the data from the csv file

In [2]:
data = pd.read_csv('data.csv', sep=',', decimal='.')
data.head()

,id,latitude,longitude,pollution,traffic_level,wind_speed,pressure,temp
0,9000000069,51.163864,4.828185,3.842857,62.32,7.20,1026.0,273.02
1,9000005034,51.130837,4.562922,10.712200,306.04,5.14,1028.0,273.10
2,9000000837,51.197985,4.478805,4.924336,129.32,7.20,1026.0,273.02
3,9000006159,51.235718,4.435498,13.448654,380.62,5.14,990.0,282.12
4,9000001420,51.162048,4.605056,3.481111,0.00,4.63,1027.0,272.57


## Insert a timestamp

The data from the Azure database doesn't have a timestamp, which is a very important element for the timeseries forecasting that we will be using. So, we will create our own timestamp by taking the current time and subtracting it by 1 hour to give us an understanding of the evolution of the data.

In [6]:
current_time = datetime.datetime.now()
periods = len(data)

start_time = current_time - pd.Timedelta(minutes=5 * (periods - 1))
date_range_subtract = pd.date_range(start=start_time, periods=periods, freq='60T')

data['timestamp'] = date_range_subtract
data.set_index('timestamp', inplace=True)
data.sort_index(inplace=True)

data

C:\Users\bogda\AppData\Local\Temp\ipykernel_25492\779135651.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range_subtract = pd.date_range(start=start_time, periods=periods, freq='60T')


,id,latitude,longitude,pollution,traffic_level,wind_speed,pressure,temp
timestamp,,,,,,,,
2024-05-03 16:21:40.902196,9000000069,51.163864,4.828185,3.842857,62.32,7.20,1026.0,273.02
2024-05-03 17:21:40.902196,9000005034,51.130837,4.562922,10.712200,306.04,5.14,1028.0,273.10
2024-05-03 18:21:40.902196,9000000837,51.197985,4.478805,4.924336,129.32,7.20,1026.0,273.02
2024-05-03 19:21:40.902196,9000006159,51.235718,4.435498,13.448654,380.62,5.14,990.0,282.12
2024-05-03 20:21:40.902196,9000001420,51.162048,4.605056,3.481111,0.00,4.63,1027.0,272.57
...,...,...,...,...,...,...,...,...
2027-11-16 00:21:40.902196,9000001873,51.062829,4.360969,18.838140,0.00,5.66,1002.0,281.42
2027-11-16 01:21:40.902196,9000003840,51.205491,4.467687,5.735327,158.01,3.60,1014.0,277.10
2027-11-16 02:21:40.902196,9000002881,51.132341,4.472705,9.878596,0.00,2.57,1009.0,277.37


## Handle missing values

In [7]:
data.fillna(data.mean(), inplace=True)
data.isnull().sum()

id               0
latitude         0
longitude        0
pollution        0
traffic_level    0
wind_speed       0
pressure         0
temp             0
dtype: int64

## Remove duplicates and outliers

To make sure the data is clean, and that there is not skewed analysis we will remove any duplicates and outliers.

In [8]:
data.drop_duplicates(inplace=True)
data.duplicated().sum()

0

## Save the clean data to a new CSV file

In [9]:
data.to_csv('clean-data.csv')